#### Implementing Apriori algorithm using python, to optimize sales in a grocery store in the south of France

Before that we need to import a teh Apyoti class from the Apyori notebook using the function and the class below

### Importing the Apyori class
#### for more infos please check: http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Importing%20Notebooks.ipynb

In [17]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [18]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [19]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


## Applying methods above to import Apyori class

##### first check if any file with this name exists

In [20]:
find_notebook("Apyori_Class_Python", path=None)

'Apyori_Class_Python.ipynb'

##### load the the module notebook to memory

In [21]:
aprio=NotebookLoader() ##self=aprio
NotebookLoader.load_module(aprio,"Apyori_Class_Python")

importing Jupyter notebook from Apyori_Class_Python.ipynb


<module 'Apyori_Class_Python' from 'Apyori_Class_Python.ipynb'>

##### importing the module as any other module

In [22]:
import Apyori_Class_Python

In [23]:
from Apyori_Class_Python import apriori

## now working on data

In [24]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)

In [27]:
dataset.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
len(dataset)

7501

each line represents a transaction of a customer
##### the apriori algo expects a list of lists, list of transactions each transaction in a list, apriori expects strings

In [29]:
transactions = []  ## list of transactions

for i in range(0, 7501):
    ## each transaction[i] in a list .values to get the values from the (i,j) cell
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

In [34]:
transactions[0]

['shrimp',
 'almonds',
 'avocado',
 'vegetables mix',
 'green grapes',
 'whole weat flour',
 'yams',
 'cottage cheese',
 'energy drink',
 'tomato juice',
 'low fat yogurt',
 'green tea',
 'honey',
 'salad',
 'mineral water',
 'salmon',
 'antioxydant juice',
 'frozen smoothie',
 'spinach',
 'olive oil']

### Training the apriori on dataset

In [44]:
### call the apriori function imported from the Apyori_Class_Python (apyori is made for apriori algorithm)
rules = apriori(transactions, min_support = 0.02, min_confidence = 0.04, min_lift = 2, min_length = 2)

### Visualising the results

In [45]:
results = list(rules)

In [46]:
results

[RelationRecord(items=frozenset({'spaghetti', 'ground beef'}), support=0.03919477403012932, ordered_statistics=[OrderedStatistic(items_base=frozenset({'ground beef'}), items_add=frozenset({'spaghetti'}), confidence=0.3989145183175034, lift=2.291162176033379), OrderedStatistic(items_base=frozenset({'spaghetti'}), items_add=frozenset({'ground beef'}), confidence=0.22511485451761104, lift=2.2911621760333793)]),
 RelationRecord(items=frozenset({'nan', 'spaghetti', 'ground beef'}), support=0.03919477403012932, ordered_statistics=[OrderedStatistic(items_base=frozenset({'nan', 'ground beef'}), items_add=frozenset({'spaghetti'}), confidence=0.3989145183175034, lift=2.291162176033379), OrderedStatistic(items_base=frozenset({'nan', 'spaghetti'}), items_add=frozenset({'ground beef'}), confidence=0.22511485451761104, lift=2.2911621760333793)]),
 RelationRecord(items=frozenset({'olive oil', 'nan', 'spaghetti'}), support=0.022930275963204905, ordered_statistics=[OrderedStatistic(items_base=frozenset